In [19]:
# Import modules.
import datetime
import numpy as np
import random
from random import shuffle
from geopy.distance import vincenty

from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Statistics
from pyevolve import Mutators
from pyevolve import Selectors

from PIL import Image, ImageDraw

In [20]:
# GPS position of each city.
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

In [28]:
LAT_MIN = 14
LAT_MAX = 26
LON_MIN = 92
LON_MAX = 99

IMG_W = 200
IMG_H = 200
IMG_BG = 255

def get_x_pos(lat):
    return int((lat - LAT_MIN) / (LAT_MAX - LAT_MIN) * IMG_W)

def get_y_pos(lon):
    return int((lon - LON_MIN) / (LON_MAX - LON_MIN) * IMG_H)

def generate_image(coords_t, path):
    data = [(IMG_BG, IMG_BG, IMG_BG) for i in range(IMG_W * IMG_H)]
    img = Image.new('RGB', (IMG_W, IMG_H))
    img.putdata(data)
    
    draw = ImageDraw.Draw(img)
    for i in range(len(path)):
        j = (i + 1) % len(path)
        draw.line(coords_t[i] + coords_t[j], fill=250)
    for i in range(len(coords_t)):
        draw.point(coords_t[i], fill=0)
    del draw
    
    return img

coords_t = []
for i in range(len(LAT)):
    coords_t.append((get_x_pos(LAT[i]), get_y_pos(LON[i])))
    
path = [1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9, 0]
img = generate_image(coords_t, path)
img.show()

In [2]:
# Fitness function.
# It simply calculates the distance in kilometers of the current path.
def fitness(chromosome):
    score = 0
    for i in range(len(chromosome)):
        j = (i + 1) % len(chromosome)
        score += matrix[chromosome[i], chromosome[j]]
    return score

In [20]:
P1 = [11, 0, 6, 3, 4, 13, 12, 9, 10, 8, 1, 5, 7, 2]
P2 = [4, 1, 10, 6, 13, 3, 2, 7, 8, 12, 0, 5, 9, 11]

def xover(P1, P2):
    assert len(P1) == len(P2)
    
    O1 = []
    O2 = []
    
    st1 = 0
    st2 = 0
    
    indexes = []
    
    # Step 2.
    O1.append(P2[st1])
    
    while (st2 < len(P1)):
        i1 = P1.index(O1[st1])
        idx = P1.index(P2[i1])
        O2.append(P2[idx])
        st1 += 1
        i2 = P1.index(O2[st2])
        indexes.append(i2)
        
        if (st1 >= len(P1)):
            break
        
        if (O2[st2] == P1[0]):
            print('O1:', O1)
            print('O2:', O2)
            nP1 = np.delete(P1, indexes).tolist()
            nP2 = np.delete(P2, indexes).tolist()
            print('nP1:', nP1)
            print('nP2:', nP2)
            print('indexes:', indexes)
            [nO1, nO2] = xover(nP1, nP2)
            print('nO1:', nO1)
            print('nO2:', nO2)
            O1.extend(nO1)
            O2.extend(nO2)
            break

        O1.append(P2[i2])
        st2 += 1
        
    return [O1, O2]

children = xover(P1, P2)

for child in children:
    print(child)

O1: [4, 6, 12]
O2: [3, 8, 11]
nP1: [0, 6, 4, 13, 12, 9, 10, 1, 5, 7, 2]
nP2: [1, 10, 13, 3, 2, 7, 8, 0, 5, 9, 11]
indexes: [3, 9, 0]
O1: [1, 0]
O2: [1, 0]
nP1: [6, 4, 13, 12, 9, 10, 5, 7, 2]
nP2: [10, 13, 3, 2, 7, 8, 5, 9, 11]
indexes: [7, 0]


ValueError: 8 is not in list

In [4]:
# Crossover Operator CX2
def G1DListCX2(genome, **args):
    gDad = args['dad'].clone()
    gMom = args['mom'].clone()
    P1 = gDad.genomeList
    P2 = gMom.genomeList
    
    print('P1:', P1)
    print('P2:', P2)
    
    O = xover(P1, P2)
        
    C1 = gDad.clone()
    C2 = gMom.clone()
    C1.resetStats()
    C2.resetStats()
    
    C1.genomeList = O[0]
    C2.genomeList = O[1]
    
    print('C1:', C1)
    print('C2:', C2)
        
    return C1, C2

In [5]:
# Callback called each time for each generation.
def tsp_callback(ga_engine):
    stats.append(ga.getStatistics().asTuple())
    return False

In [6]:
# Initializator for the 1DList.
# Simply creates a list containing the cities indexes and shuffle it.
def list_initializator(genome, **args):
    cities = [i for i in range(len(coords))]
    random.shuffle(cities)
    genome.genomeList = cities
    #   genome.setInternalList(cities)


In [7]:
# Save the GA statistics in `filename`.
def save_stats(filename, stats):
    header = 'rawMax;rawMin;rawAve;rawDev;rawVar;fitMax;fitMin;fitAve'
    np.savetxt(filename, stats, '%.2f', delimiter=';', header=header, comments='')
    print('Saved stats into ' + filename)

In [8]:
# Create a 1D list of tuples with latitude and longitude for each city.
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))

# Create a distance matrix between each city.
matrix = {}
for i, city1 in enumerate(coords):
    for j, city2 in enumerate(coords):
        dist = vincenty(city1, city2).km
        matrix[i, j] = dist
        
# Constants.
POPULATION_SIZE = 100
MUTATION_RATE   = 0.01
CROSSOVER_RATE  = 0.9
GENERATIONS_NB  = 300
CROSSOVER_FUNC  = G1DListCX2
#CROSSOVER_FUNC  = Crossovers.G1DListCrossoverEdge

# Array for collecting statistics.
stats = []

# The chromosome is represented by a 1DList which contains the cities indexes [0,13]
genome = G1DList.G1DList(len(coords))
# Set fitness function.
genome.evaluator.set(fitness)
# Set crossover function.
genome.crossover.set(CROSSOVER_FUNC)
# Set initializator function.
genome.initializator.set(list_initializator)

# GA initialization.
ga = GSimpleGA.GSimpleGA(genome)

ga.setPopulationSize(POPULATION_SIZE)
ga.setMutationRate(MUTATION_RATE)

ga.setCrossoverRate(CROSSOVER_RATE)
# Elitism is important for TSP.
ga.selector.set(Selectors.GTournamentSelector)

ga.setElitism(True)
# We aim for minimization.
ga.setMinimax(Consts.minimaxType["minimize"])

# Number of generations
ga.setGenerations(GENERATIONS_NB)

# In case we want to monitor the evolution process
# execute the function current_best every generation
ga.stepCallback.set(tsp_callback)

ga.evolve(freq_stats=500)

# Final best solution
print(ga.bestIndividual())

# Save statistics.
if CROSSOVER_FUNC == Crossovers.G1DListCrossoverEdge:
    crossover_name = 'edge'
else:
    crossover_name = 'perso'
    
now = datetime.datetime.now()

save_stats('stats/' +
               crossover_name +
               '_pop' + str(POPULATION_SIZE) +
               '_gen' + str(GENERATIONS_NB) +
               '_' + now.strftime('%d%m%Y-%H%M') +
               '.csv',
           stats)

# Best path found: [1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9, 0]
# For crossovers inspiration: https://www.hindawi.com/journals/cin/2017/7430125/

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [7838.89(8382.91)/5474.19(5033.54)/6532.40(6532.40)]
P1: [11, 0, 6, 3, 4, 13, 12, 9, 10, 8, 1, 5, 7, 2]
P2: [4, 1, 10, 6, 13, 3, 2, 7, 8, 12, 0, 5, 9, 11]


ValueError: 4 is not in list